In [18]:
import pickle
import pandas as pd
import cv2
import os
import imutils
import numpy as np
import time
from keras.models import load_model
import glob

In [24]:
videoAddr = '../testVideos/3135.mp4'
tempFolder = '../temp'
modelPathPFR = '../Models/PFRModel/PFRModel.sav'
modelPathFT = '../Models/FuelTypeModel/FTModel.sav'

In [25]:
PFRCLASSES = ['0', '1-3', '10-12', '13-15', '20-25', '30-40', '4-5', '41-56', '6-9', '60+']
FTCLASSES = ['20H280NG', 'Ethlyne', 'F1', 'F2', 'NG']

In [26]:
def classifyPFR(model,image):
    # load the input image and then clone it so we can draw on it later
    # our model was trained on RGB ordered images but OpenCV represents
    # images in BGR order, so swap the channels, and then resize to
    # 224x224 (the input dimensions for VGG16)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    # convert the image to a floating point data type and perform mean
    # subtraction
    image = image.astype("float32")
    mean = np.array([123.68, 116.779, 103.939][::-1], dtype="float32")
    image -= mean
    
    # pass the image through the network to obtain our predictions
    preds = model.predict(np.expand_dims(image, axis=0))[0]
    
    return preds

In [27]:
def classifyFT(model,image):
    # load the input image and then clone it so we can draw on it later
    # our model was trained on RGB ordered images but OpenCV represents
    # images in BGR order, so swap the channels, and then resize to
    # 224x224 (the input dimensions for VGG16)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    # convert the image to a floating point data type and perform mean
    # subtraction
    image = image.astype("float32")
    mean = np.array([123.68, 116.779, 103.939][::-1], dtype="float32")
    image -= mean
    
    # pass the image through the network to obtain our predictions
    preds = model.predict(np.expand_dims(image, axis=0))[0]
    
    return preds

In [28]:
if not os.path.exists(tempFolder):
    os.makedirs(tempFolder)
else:
    # already exixts, clean it
    files = glob.glob(tempFolder+'/*')
    for f in files:
        os.remove(f)
import pdb
#pdb.set_trace()
vidcap = cv2.VideoCapture(videoAddr)
success,image = vidcap.read()
count = 0

print("[INFO] loading model...")
#pdb.set_trace()
modelPFR = pickle.load(open(modelPathPFR, 'rb'))
modelFT = pickle.load(open(modelPathFT, 'rb'))
while success:
    output = image.copy()
    output = imutils.resize(output, width=400)
    preds = classifyPFR(modelPFR, image)
    ft = classifyFT(modelFT, image)

    # draw the prediction on the output image
    pfr_i = np.argmax(preds)
    ft_i = np.argmax(ft)
    pfrlabel = PFRCLASSES[pfr_i]
    ftlabel = FTCLASSES[ft_i]
    pfrtext = "PFR - {pfr}: {pfrProb}%".format(pfr =pfrlabel, pfrProb=np.round(preds[pfr_i] * 100,2))
    fttext =  "Fuel Type - {ft}: {ftProb}%".format(ft=ftlabel,ftProb=np.round(ft[ft_i]*100,2))
    cv2.putText(output, pfrtext, (3, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0, 255, 0), 2)
    cv2.putText(output, fttext, (3, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0, 255, 0), 2)
    # show the output image
    cv2.imshow("Output", output)
    cv2.waitKey(1)

    success,image = vidcap.read()
    count += 1
cv2.destroyWindow('Output') 

[INFO] loading model...
